In [57]:
# Librerías
import requests
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from astroquery.mpc import MPC

# Cobs API

In [58]:
# Verificar la conexión a internet.
def verificar_conexion():
    try:
        requests.get("http://www.google.com", timeout=5)
        print('✅ Conectado a internet.')
        return True
    
    except requests.ConnectionError:
        print('🛑 Sin conexión a internet.')
        return False

In [59]:
# Conexión con la API de COBS
nombre_cometa =  'C/2023 A3'
Link_cops_API = f'https://cobs.si/api/obs_list.api?des={nombre_cometa}&format=json&from_date=&to_date=&exclude_faint=False&exclude_not_accurate=False'

if verificar_conexion():
    response = requests.get(Link_cops_API)


if response.status_code == 200:
    content = response.json()
    print('✅ Base de datos actualizada.')

else:
    print(f'🛑 Se presentó un error al cargar la base de datos.\nError: {response.status_code}\n{response.content}')

✅ Conectado a internet.
✅ Base de datos actualizada.


In [60]:
# Creación del data frame Cometa
cometa_df = pd.DataFrame(content['objects'])
cometa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2357 entries, 0 to 2356
Data columns (total 47 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   type                       2357 non-null   object 
 1   obs_date                   2357 non-null   object 
 2   comet                      2357 non-null   object 
 3   observer                   2357 non-null   object 
 4   location                   1216 non-null   object 
 5   extinction                 318 non-null    object 
 6   obs_method                 2356 non-null   object 
 7   comet_visibility           21 non-null     object 
 8   magnitude                  2355 non-null   object 
 9   conditions                 111 non-null    object 
 10  ref_catalog                2356 non-null   object 
 11  instrument_aperture        2357 non-null   object 
 12  instrument_type            2357 non-null   object 
 13  instrument_focal_ratio     1500 non-null   float

In [61]:
# Sin filtrar la información
filas,columnas = cometa_df.shape
print(f'Registros: {filas}\nVariables: {columnas}')

Registros: 2357
Variables: 47


In [62]:
# Base de datos arrojada por la API
cometa_df.sample(5)

,type,obs_date,comet,observer,location,extinction,obs_method,comet_visibility,magnitude,conditions,...,magnitude_error,comparison_star_magnitude,pixel_size_x,pixel_size_y,pixel_size_unit,obs_comment,obs_sky_quality,obs_sky_quality_method,reference_star_names,date_added
1598,C,2024-05-16 11:45:35,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Katsumi', 'last_name': 'Yoshim...",None,a,"{'key': 'V', 'name': 'Johnson/Bessel/Kron/Cous...",None,10.7,None,...,None,11.50,2.0,2.0,s,,None,NaN,,2024-05-23 03:24:15
1642,C,2024-05-12 22:25:00,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Mieczyslaw Leszek', 'last_name...",None,None,"{'key': 'k', 'name': 'Kron/Cousins R with CCD'...",None,10.0,None,...,0.00,None,2.9,2.9,s,Temperature: +5 C,None,NaN,,2024-05-21 18:14:12
2130,C,2024-02-25 03:07:11,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Thomas', 'last_name': 'Lehmann...",Weimar,None,"{'key': 'Z', 'name': 'CCD Visual equivalent', ...",None,13.0,None,...,None,None,1.3,1.3,s,"CMOS OSC/G, mlim=19.8, moon 100% dist 64 deg",None,NaN,,2024-03-07 16:32:57
1231,C,2024-07-29 08:38:23,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Pieter-Jan', 'last_name': 'Dek...",None,None,"{'key': 'C', 'name': 'Unfiltered total CCD mag...",None,8.9,None,...,0.07,None,1.2,1.2,s,,None,NaN,,2024-08-23 10:20:45
2113,C,2024-03-07 00:43:12,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Pieter-Jan', 'last_name': 'Dek...",None,None,"{'key': 'C', 'name': 'Unfiltered total CCD mag...",None,12.3,None,...,0.07,None,1.1,1.1,s,,None,NaN,,2024-03-10 09:48:03


In [63]:
cometa_df.obs_method.apply(
    lambda registro: f"{registro['key']}: {registro['name']}" if (registro is not None) and ('key' in registro) and ('name' in registro) else 'Datos faltantes'
).value_counts()

obs_method
C: Unfiltered total CCD magnitude                                                   566
M: Modified-Out method                                                              503
S: In-Out method                                                                    403
Z: CCD Visual equivalent                                                            339
B: Simple Out-Out method                                                            150
V: Johnson/Bessel/Kron/Cousins V with CCD                                           108
I: In-focus                                                                         104
k: Kron/Cousins R with CCD                                                           44
D: Johnson/Bessel/Kron/Cousins B with CCD                                            27
H: Kron/Cousins I with CCD                                                           21
-: Unknown                                                                           18
P: photographic      

In [64]:
# Tratamiento de los datos de interés
cometa_df['obs_method_key'] = cometa_df.obs_method.apply(lambda registro: registro['key'] if registro is not None and 'key' in registro else 'Dato faltante')
cometa_df['obs_date'] = pd.to_datetime(pd.to_datetime(cometa_df.obs_date).dt.date)
cometa_df['magnitude'] = pd.to_numeric(cometa_df.magnitude)

In [65]:
# Creación del data frame curva de luz cruda
curva_de_luz_cruda_df = cometa_df[['obs_method_key', 'obs_date', 'magnitude']].copy()
curva_de_luz_cruda_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2357 entries, 0 to 2356
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   obs_method_key  2357 non-null   object        
 1   obs_date        2357 non-null   datetime64[ns]
 2   magnitude       2355 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 55.4+ KB


In [66]:
# Con la información filtrada
filas,columnas = curva_de_luz_cruda_df.shape
print(f'Registros: {filas}\nVariables: {columnas}')

Registros: 2357
Variables: 3


In [67]:
# Data Frame de la curva de luz
curva_de_luz_cruda_df.sample(5)

,obs_method_key,obs_date,magnitude
731,S,2024-10-16,3.0
355,S,2024-10-28,5.0
432,-,2024-10-24,5.2
2142,S,2024-02-14,12.8
1375,S,2024-06-19,10.6


In [68]:
# Curva de luz cruda.
labels = {'obs_date':'Observation Date','magnitude':'Apparent total magnitude', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_cruda_df, x='obs_date', y='magnitude', color='obs_method_key', template= 'plotly_dark', labels= labels, title= f'Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

# MPC API usando astroquery.

In [69]:
# Creación de data frame Ephemeris (conexión con la API del MPC)
fecha_inicial = curva_de_luz_cruda_df.obs_date.min()
fecha_final = curva_de_luz_cruda_df.obs_date.max()
fechas = (fecha_final - fecha_inicial).days + 1 if (fecha_final - fecha_inicial).days <= 1441 else 1441

ephemeris = MPC.get_ephemeris(nombre_cometa, start = str(fecha_inicial), number = fechas)  # type: ignore

ephemeris_df = ephemeris.to_pandas()
ephemeris_df.columns = ephemeris_df.columns.str.lower().str.replace(' ', '_')
ephemeris_df

,date,ra,dec,delta,r,elongation,phase,v,proper_motion,direction
0,2023-02-25,230.565417,-1.256111,6.946,7.290,106.6,7.5,17.6,8.17,318.9
1,2023-02-26,230.528333,-1.214722,6.921,7.281,107.6,7.4,17.6,8.51,317.4
2,2023-02-27,230.488750,-1.172778,6.896,7.272,108.6,7.4,17.6,8.86,316.0
3,2023-02-28,230.446250,-1.130000,6.872,7.264,109.6,7.4,17.6,9.21,314.7
4,2023-03-01,230.401250,-1.086389,6.847,7.255,110.6,7.3,17.6,9.56,313.4
...,...,...,...,...,...,...,...,...,...,...
641,2024-11-27,285.528750,4.346944,1.839,1.403,48.9,32.0,9.0,54.84,85.0
642,2024-11-28,285.890000,4.379444,1.869,1.421,48.4,31.3,9.1,53.69,84.7
643,2024-11-29,286.243750,4.413611,1.899,1.438,47.9,30.6,9.2,52.62,84.3
644,2024-11-30,286.590417,4.448889,1.929,1.455,47.4,29.9,9.2,51.61,84.0


In [70]:
# Info del data frame ephemeris
ephemeris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 646 entries, 0 to 645
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           646 non-null    datetime64[ns]
 1   ra             646 non-null    float64       
 2   dec            646 non-null    float64       
 3   delta          646 non-null    float64       
 4   r              646 non-null    float64       
 5   elongation     646 non-null    float64       
 6   phase          646 non-null    float64       
 7   v              646 non-null    float64       
 8   proper_motion  646 non-null    float64       
 9   direction      646 non-null    float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 50.6 KB


In [71]:
# Dar a los datos el formato deseado
ephemeris_df.date = pd.to_datetime(ephemeris_df.date)
ephemeris_df.date = pd.to_datetime(ephemeris_df.date.dt.date)
ephemeris_df.dtypes

date             datetime64[ns]
ra                      float64
dec                     float64
delta                   float64
r                       float64
elongation              float64
phase                   float64
v                       float64
proper_motion           float64
direction               float64
dtype: object

In [72]:
# Creación del data frame ephemeris filtrada
ephemeris_filtrada_df = ephemeris_df[['date', 'delta','r', 'phase']].copy()
ephemeris_filtrada_df = ephemeris_filtrada_df.rename(columns = {'date':'obs_date'})
ephemeris_filtrada_df

,obs_date,delta,r,phase
0,2023-02-25,6.946,7.290,7.5
1,2023-02-26,6.921,7.281,7.4
2,2023-02-27,6.896,7.272,7.4
3,2023-02-28,6.872,7.264,7.4
4,2023-03-01,6.847,7.255,7.3
...,...,...,...,...
641,2024-11-27,1.839,1.403,32.0
642,2024-11-28,1.869,1.421,31.3
643,2024-11-29,1.899,1.438,30.6
644,2024-11-30,1.929,1.455,29.9


# Unión de las bases de datos.

In [73]:
# Unión de las bases de datos COBS y MPC
curva_de_luz_procesada_df = curva_de_luz_cruda_df.merge(ephemeris_filtrada_df, on='obs_date')
curva_de_luz_procesada_df

,obs_method_key,obs_date,magnitude,delta,r,phase
0,M,2024-12-01,9.0,1.958,1.472,29.3
1,P,2024-12-01,9.3,1.958,1.472,29.3
2,C,2024-12-01,9.1,1.958,1.472,29.3
3,S,2024-11-30,9.0,1.929,1.455,29.9
4,S,2024-11-30,8.8,1.929,1.455,29.9
...,...,...,...,...,...,...
2352,C,2023-02-27,18.0,6.896,7.272,7.4
2353,C,2023-02-26,18.1,6.921,7.281,7.4
2354,C,2023-02-25,18.2,6.946,7.290,7.5
2355,C,2023-02-25,17.9,6.946,7.290,7.5


In [74]:
# Información del data frame curva de lus procesada
curva_de_luz_procesada_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2357 entries, 0 to 2356
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   obs_method_key  2357 non-null   object        
 1   obs_date        2357 non-null   datetime64[ns]
 2   magnitude       2355 non-null   float64       
 3   delta           2357 non-null   float64       
 4   r               2357 non-null   float64       
 5   phase           2357 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 110.6+ KB


In [75]:
# Reducción de la magnitud aparente
beta = 0

curva_de_luz_procesada_df['magnitud_reducida'] = (
    curva_de_luz_cruda_df['magnitude'] 
    - 5 * np.log10(curva_de_luz_procesada_df['delta'] * curva_de_luz_procesada_df['r'])
    - (beta * curva_de_luz_procesada_df['phase'])
    )
curva_de_luz_procesada_df

,obs_method_key,obs_date,magnitude,delta,r,phase,magnitud_reducida
0,M,2024-12-01,9.0,1.958,1.472,29.3,6.701398
1,P,2024-12-01,9.3,1.958,1.472,29.3,7.001398
2,C,2024-12-01,9.1,1.958,1.472,29.3,6.801398
3,S,2024-11-30,9.0,1.929,1.455,29.9,6.759024
4,S,2024-11-30,8.8,1.929,1.455,29.9,6.559024
...,...,...,...,...,...,...,...
2352,C,2023-02-27,18.0,6.896,7.272,7.4,9.498744
2353,C,2023-02-26,18.1,6.921,7.281,7.4,9.588201
2354,C,2023-02-25,18.2,6.946,7.290,7.5,9.677688
2355,C,2023-02-25,17.9,6.946,7.290,7.5,9.377688


In [76]:
# Curva de luz reducida
labels = {'obs_date':'Observation Date','magnitud_reducida':'Apparent total magnitude processed', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_procesada_df, x='obs_date', y='magnitud_reducida', color='obs_method_key', template= 'plotly_dark', labels= labels, title=f'Reduced Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

In [77]:
# Creación del data frame curva de luz promediada
numero_elementos_grupo = 7

curva_de_luz_promediada_df = curva_de_luz_procesada_df.copy()
curva_de_luz_promediada_df['promedio_movil'] = curva_de_luz_promediada_df.magnitud_reducida.rolling(window = numero_elementos_grupo).mean()
curva_de_luz_promediada_df

,obs_method_key,obs_date,magnitude,delta,r,phase,magnitud_reducida,promedio_movil
0,M,2024-12-01,9.0,1.958,1.472,29.3,6.701398,NaN
1,P,2024-12-01,9.3,1.958,1.472,29.3,7.001398,NaN
2,C,2024-12-01,9.1,1.958,1.472,29.3,6.801398,NaN
3,S,2024-11-30,9.0,1.929,1.455,29.9,6.759024,NaN
4,S,2024-11-30,8.8,1.929,1.455,29.9,6.559024,NaN
...,...,...,...,...,...,...,...,...
2352,C,2023-02-27,18.0,6.896,7.272,7.4,9.498744,9.500592
2353,C,2023-02-26,18.1,6.921,7.281,7.4,9.588201,9.507435
2354,C,2023-02-25,18.2,6.946,7.290,7.5,9.677688,9.500014
2355,C,2023-02-25,17.9,6.946,7.290,7.5,9.377688,9.494068


In [78]:
# Curva de luz Promediada
labels = {'obs_date':'Observation Date','magnitud_reducida':'Max apparent total magnitude reduced', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_promediada_df, x='obs_date', y='promedio_movil', color='obs_method_key', template= 'plotly_dark', labels= labels, title= f'Average Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz interna (Envolvente inferior) v1 (Promedio corrido -> agrupación)

In [79]:
# Creación del data frame curva de luz agrupada
curva_de_luz_interna_v1_df = curva_de_luz_promediada_df.groupby(by = 'obs_date').max()
curva_de_luz_interna_v1_df = curva_de_luz_interna_v1_df.reset_index()
curva_de_luz_interna_v1_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,promedio_movil
0,2023-02-25,C,18.3,6.946,7.290,7.5,9.777688,9.503923
1,2023-02-26,C,18.1,6.921,7.281,7.4,9.588201,9.507435
2,2023-02-27,C,18.0,6.896,7.272,7.4,9.498744,9.500592
3,2023-02-28,C,18.2,6.872,7.264,7.4,9.708705,9.585723
4,2023-03-01,Z,17.9,6.847,7.255,7.3,9.419311,9.552207
...,...,...,...,...,...,...,...,...
375,2024-11-27,k,9.6,1.839,1.403,32.0,7.541803,6.787957
376,2024-11-28,S,9.0,1.869,1.421,31.3,6.878983,6.627497
377,2024-11-29,Z,8.8,1.899,1.438,30.6,6.618581,6.524675
378,2024-11-30,S,9.0,1.929,1.455,29.9,6.759024,6.691470


In [80]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_interna_v1_df, x='obs_date', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='red', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz Externa (Envolvente superior) v1 (Promedio corrido -> agrupación)

In [81]:
# Creación del data frame curva de luz agrupada
curva_de_luz_externa_v1_df = curva_de_luz_promediada_df.groupby(by = 'obs_date').min()
curva_de_luz_externa_v1_df = curva_de_luz_externa_v1_df.reset_index()
curva_de_luz_externa_v1_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,promedio_movil
0,2023-02-25,C,17.9,6.946,7.290,7.5,9.377688,9.494068
1,2023-02-26,C,18.1,6.921,7.281,7.4,9.588201,9.507435
2,2023-02-27,C,17.9,6.896,7.272,7.4,9.398744,9.495630
3,2023-02-28,C,17.7,6.872,7.264,7.4,9.208705,9.504953
4,2023-03-01,Z,17.9,6.847,7.255,7.3,9.419311,9.552207
...,...,...,...,...,...,...,...,...
375,2024-11-27,C,8.1,1.839,1.403,32.0,6.041803,6.656089
376,2024-11-28,B,8.3,1.869,1.421,31.3,6.178983,6.435839
377,2024-11-29,S,8.4,1.899,1.438,30.6,6.218581,6.444422
378,2024-11-30,B,8.5,1.929,1.455,29.9,6.259024,6.593649


In [82]:
# Gráfica de lus promediada
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_externa_v1_df, x='obs_date', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Max averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='yellow', size=6, line= dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz interna (Envolvente inferior) v2 (Agrupación  -> promedio corrido)

In [83]:
# Creación del data frame curva de luz agrupada
curva_de_luz_agrupada_max_v2_df = curva_de_luz_procesada_df.groupby(by = 'obs_date').max()
curva_de_luz_agrupada_max_v2_df = curva_de_luz_agrupada_max_v2_df.reset_index()
curva_de_luz_agrupada_max_v2_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida
0,2023-02-25,C,18.3,6.946,7.290,7.5,9.777688
1,2023-02-26,C,18.1,6.921,7.281,7.4,9.588201
2,2023-02-27,C,18.0,6.896,7.272,7.4,9.498744
3,2023-02-28,C,18.2,6.872,7.264,7.4,9.708705
4,2023-03-01,Z,17.9,6.847,7.255,7.3,9.419311
...,...,...,...,...,...,...,...
375,2024-11-27,k,9.6,1.839,1.403,32.0,7.541803
376,2024-11-28,S,9.0,1.869,1.421,31.3,6.878983
377,2024-11-29,Z,8.8,1.899,1.438,30.6,6.618581
378,2024-11-30,S,9.0,1.929,1.455,29.9,6.759024


In [84]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_agrupada_max_v2_df, x='obs_date', y='magnitud_reducida', template= 'plotly_dark',color='obs_method_key', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

In [85]:
# Creación del data frame curva de luz promediada
numero_elementos_grupo = 7

curva_de_luz_interna_v2_df = curva_de_luz_agrupada_max_v2_df.copy()
curva_de_luz_interna_v2_df['promedio_movil'] = curva_de_luz_interna_v2_df.magnitud_reducida.rolling(window = numero_elementos_grupo).mean()
curva_de_luz_interna_v2_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,promedio_movil
0,2023-02-25,C,18.3,6.946,7.290,7.5,9.777688,NaN
1,2023-02-26,C,18.1,6.921,7.281,7.4,9.588201,NaN
2,2023-02-27,C,18.0,6.896,7.272,7.4,9.498744,NaN
3,2023-02-28,C,18.2,6.872,7.264,7.4,9.708705,NaN
4,2023-03-01,Z,17.9,6.847,7.255,7.3,9.419311,NaN
...,...,...,...,...,...,...,...,...
375,2024-11-27,k,9.6,1.839,1.403,32.0,7.541803,7.353289
376,2024-11-28,S,9.0,1.869,1.421,31.3,6.878983,7.258296
377,2024-11-29,Z,8.8,1.899,1.438,30.6,6.618581,7.064713
378,2024-11-30,S,9.0,1.929,1.455,29.9,6.759024,7.001250


In [86]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_interna_v2_df, x='obs_date', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='red', size=5, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz Externa (Envolvente superior) v2 (Agrupación  -> promedio corrido)

In [87]:
# Creación del data frame curva de luz agrupada
curva_de_luz_agrupada_min_v2_df = curva_de_luz_procesada_df.groupby(by = 'obs_date').min()
curva_de_luz_agrupada_min_v2_df = curva_de_luz_agrupada_min_v2_df.reset_index()
curva_de_luz_agrupada_min_v2_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida
0,2023-02-25,C,17.9,6.946,7.290,7.5,9.377688
1,2023-02-26,C,18.1,6.921,7.281,7.4,9.588201
2,2023-02-27,C,17.9,6.896,7.272,7.4,9.398744
3,2023-02-28,C,17.7,6.872,7.264,7.4,9.208705
4,2023-03-01,Z,17.9,6.847,7.255,7.3,9.419311
...,...,...,...,...,...,...,...
375,2024-11-27,C,8.1,1.839,1.403,32.0,6.041803
376,2024-11-28,B,8.3,1.869,1.421,31.3,6.178983
377,2024-11-29,S,8.4,1.899,1.438,30.6,6.218581
378,2024-11-30,B,8.5,1.929,1.455,29.9,6.259024


In [88]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_agrupada_min_v2_df, x='obs_date', y='magnitud_reducida', template= 'plotly_dark',color='obs_method_key', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

In [89]:
# Creación del data frame curva de luz promediada
numero_elementos_grupo = 7

curva_de_luz_externa_v2_df = curva_de_luz_agrupada_min_v2_df.copy()
curva_de_luz_externa_v2_df['promedio_movil'] = curva_de_luz_externa_v2_df.magnitud_reducida.rolling(window = numero_elementos_grupo).mean()
curva_de_luz_externa_v2_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,promedio_movil
0,2023-02-25,C,17.9,6.946,7.290,7.5,9.377688,NaN
1,2023-02-26,C,18.1,6.921,7.281,7.4,9.588201,NaN
2,2023-02-27,C,17.9,6.896,7.272,7.4,9.398744,NaN
3,2023-02-28,C,17.7,6.872,7.264,7.4,9.208705,NaN
4,2023-03-01,Z,17.9,6.847,7.255,7.3,9.419311,NaN
...,...,...,...,...,...,...,...,...
375,2024-11-27,C,8.1,1.839,1.403,32.0,6.041803,6.181861
376,2024-11-28,B,8.3,1.869,1.421,31.3,6.178983,6.158296
377,2024-11-29,S,8.4,1.899,1.438,30.6,6.218581,6.178999
378,2024-11-30,B,8.5,1.929,1.455,29.9,6.259024,6.215536


In [90]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_externa_v2_df, x='obs_date', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Max Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='yellow', size=5, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz usando la mediada.

In [ ]:
# # Creación del data frame curva de luz mediana
curva_de_luz_mediada_v1_df = curva_de_luz_procesada_df.groupby(by= 'obs_date').median(numeric_only= True)
curva_de_luz_mediada_v1_df = curva_de_luz_mediada_v1_df.reset_index()
curva_de_luz_mediada_v1_df

,obs_date,magnitude,delta,r,phase,magnitud_reducida
0,2023-02-25,18.20,6.946,7.290,7.5,9.677688
1,2023-02-26,18.10,6.921,7.281,7.4,9.588201
2,2023-02-27,17.95,6.896,7.272,7.4,9.448744
3,2023-02-28,17.95,6.872,7.264,7.4,9.458705
4,2023-03-01,17.90,6.847,7.255,7.3,9.419311
...,...,...,...,...,...,...
375,2024-11-27,8.60,1.839,1.403,32.0,6.541803
376,2024-11-28,8.85,1.869,1.421,31.3,6.728983
377,2024-11-29,8.60,1.899,1.438,30.6,6.418581
378,2024-11-30,8.80,1.929,1.455,29.9,6.559024


In [173]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_mediada_v1_df, x='obs_date', y='magnitud_reducida', template= 'plotly_dark', labels= labels, title=f'Mediated Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

# Comparación de las curvas de luz v1 (Promedio corrido -> agrupación)

In [168]:
# Gráfica de luz promediada
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = go.Figure()
fig.add_trace(go.Scatter(x=curva_de_luz_externa_v1_df.obs_date, y=curva_de_luz_externa_v1_df.promedio_movil, mode='markers', name='Envolvente', marker=dict(color='yellow', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_v1_df.obs_date, y=curva_de_luz_interna_v1_df.promedio_movil, mode='markers', name='Núcleo', marker=dict(color='red', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_mediada_v1_df.obs_date, y=curva_de_luz_mediada_v1_df.magnitud_reducida, mode='markers', name='Mediana', marker=dict(color='light blue', line=dict(width=1, color='DarkSlateGrey'))))
fig.update_layout(template='plotly_dark')
fig.update_yaxes(autorange="reversed")
fig.update_layout(template='plotly_dark', xaxis_title='Observation Date', yaxis_title='Averaged Magnitude', title = f'Max/Min Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.show()

# Comparación de las curvas de luz v2 (Agrupación -> promedio corrido)

In [94]:
# Gráfica de luz promediada
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = go.Figure()
fig.add_trace(go.Scatter(x=curva_de_luz_externa_v2_df.obs_date, y=curva_de_luz_externa_v2_df.promedio_movil, mode='markers', name='Envolvente', marker=dict(color='yellow', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_v2_df.obs_date, y=curva_de_luz_interna_v2_df.promedio_movil, mode='markers', name='Núcleo', marker=dict(color='red', line=dict(width=1, color='DarkSlateGrey'))))
fig.update_layout(template='plotly_dark')
fig.update_yaxes(autorange="reversed")
fig.update_layout(template='plotly_dark', xaxis_title='Observation Date', yaxis_title='Averaged Magnitude', title = f'Max/Min Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.show()